In [8]:
import requests
import os
import re
import time
import pandas as pd
from bs4  import BeautifulSoup as bs
import urllib.request
import numpy as np
import sys

In [2]:
# We give the url of the page to scrap and the tag that contains the data that interests us

def extractData(url,balise):
  page=urllib.request.urlopen(url)  
  data= bs(page)  # To load the page code

  # Extracting data from the tag to put them in the same table
  return data.find(balise)

In [3]:
def getData(termEnt, relationEnt):

    a = termEnt
    r = relationEnt

    # In case when the term does not exist
    if os.path.isfile(f"{a}_cache.txt"):
        sys.exit(f"Ce terme '{a}' a été déjà cherché et jugé none existante. ")  


    url = f'http://www.jeuxdemots.org/rezo-dump.php?gotermsubmit=Chercher&gotermrel={a}&rel={r}'
    
    try:
        data = extractData(url,'code').getText()

    except:
        open(f"{a}_cache.txt", "x")
        sys.exit(f"Le terme {a} n'existe pas.")

    
     ########### PREPROCESSING  ############

    comments = "^\/\/ *.*"
    start_Ent = "(?s)^.*?(?=e;eid)"        # What's before the first node
    start_Rel = "(?s)^.*?(?=r;rid)"
    empty_lines = "^\s*$"
    quotes = r"(?!\b'\b)'"

    # ENTRIES 
    e1 = re.sub(start_Ent, '', data) 
    e2 = re.sub(comments, '', e1, flags=re.MULTILINE)  
    e3 = re.sub("((r;.*)|(rt;.*))", '', e2) 
    e4 = re.sub(empty_lines, '', e3, flags=re.MULTILINE) 
    entries = re.sub(quotes, '', e4)

    # RELATIONS 
    r1 = re.sub(start_Rel, '',  data)
    relations = re.sub(comments, '', r1, flags=re.MULTILINE)

    # To make object as list
    entries = entries.split("\n")
    relations = relations.split("\n")


    fileForEntry = f"{a}_Ent.csv"
    fileForRelation = f"{a}_Rel.csv"

    # Convert raw data into semi-structured data in csv format file and save it.
    np.savetxt(fileForEntry, entries, delimiter =",", fmt ='%s')
    np.savetxt(fileForRelation, relations, delimiter =",", fmt ='%s')

    # df for entries
    df_ent=pd.read_csv(fileForEntry, on_bad_lines='skip', delimiter=';')

    # df for relations
    df_rel=pd.read_csv(fileForRelation, on_bad_lines='skip', delimiter=';')
 
    return df_ent, df_rel

In [4]:
def getTerm(termEnt, data):
  search = re.search("(?<=>)[0-9]*", data[0])
  id = search.group()
  rowTerm = termEnt.loc[termEnt['eid'] == int(id)]

  return "(" + rowTerm['name'].values[0] + ")"

In [5]:
def getFile(f):
  f_Ent = f"{f}_Ent.csv"
  f_Rel = f"{f}_Relt.csv"

  if (os.path.exists(f_Ent)) & (os.path.exists(f_Rel)):
    return True
  else:
    return False

In [6]:
def getInference(a,r,b):

  p = " \n Quels types d'inférence souhaitez_vous produire ? \n \
        1. Inférence déductive  \n \
        2. Inférence inductive  \n \
        3. Inférence transitive \n \
        \n \
        "
  inference_type = input(p)

  while (inference_type not in ['1','2','3']):
      inferenceType = input("Veuillez choisir parmi les options proposées svp: \n")

  counter = 0

  ############  DEDUCTION   ####################

  if (inference_type =='1'):

    dfTerm, dfRel = getData(a,"r_isa")
    dfRel= dfRel.sort_values(by='w ', ascending=False)

    IdTerme_2 = np.array(dfRel.node2.values)
    names = dfTerm.set_index("eid").loc[IdTerme_2, ['name']].values.flatten()

    for x in names :

      fileName = getFile(x)

      start = time.time()
      print("\nRecherche,... - mot trouvé : ", x)

      if x==b :
        print("\n", a, r, b," => Oui, car", a, r, x, "et", x, r, b)
        return

      try:

        if (fileName==True):
          print("Le term existe déjà, trouvons un résultat...\n")
          newDfEnt = pd.read_csv(f_Ent, on_bad_lines='skip', delimiter=';')
          newDfRel = pd.read_csv(f_Rel, on_bad_lines='skip', delimiter=';')

        else:
          print("Le term n'existe pas, essayons de le chercher...\n")
          newDfEnt, newDfRel = getData(x, r)

        
        newDfRel = newDfRel.sort_values(by='w ', ascending=False)
        IdTerme_22 = np.array(newDfRel.node2.values)
        names2 = newDfEnt.set_index("eid").loc[IdTerme_22, ['name']].values.flatten()

        end = time.time()
        timer = end-start
        counter += timer

        if(counter >= 5):

          print("\n Le programme s'est arrété")
          break

        if b in names2:

          if r == "r_has_part":    # Inversion
            a,b = b,a

          if '>' in x:
            dta = getTerm(dfTerm, names)
            x = re.sub("[0-9].*", dta, x)

          print("\n", a, r, b," => Oui, car", a, "r_isa" , x, " et ", x, r, b)
          return

      except:
        pass  

    print(a,r,b,"=>  Aucune inférence déductive trouvée, veuillez avec les autres options. ")
    return


  ############  INDUCTION   ####################

  if (inference_type =='2'):

    dfTerm, dfRel = getData(a,"r_hypo")
    dfRel= dfRel.sort_values(by='w ', ascending=False)

    IdTerme_2 = np.array(dfRel.node2.values)
    names = dfTerm.set_index("eid").loc[IdTerme_2, ['name']].values.flatten()

    for x in names :

      fileName = getFile(x)

      start = time.time()
      print("\nRecherche,... - mot trouvé : ", x)

      if x==b :
        print("\n", a, r, b," => Oui, car", a, r, x, "et", x, r, b)
        return

      try:

        if (fileName==True):
          print("Le term existe déjà, trouvons un résultat...\n")
          newDfEnt = pd.read_csv(f_Ent, on_bad_lines='skip', delimiter=';')
          newDfRel = pd.read_csv(f_Rel, on_bad_lines='skip', delimiter=';')

        else:
          print("Le term n'existe pas, essayons de le chercher...\n")
          newDfEnt, newDfRel = getData(x, r)

        
        newDfRel = newDfRel.sort_values(by='w ', ascending=False)
        IdTerme_22 = np.array(newDfRel.node2.values)
        names2 = newDfEnt.set_index("eid").loc[IdTerme_22, ['name']].values.flatten()

        end = time.time()
        timer = end-start
        counter += timer

        if(counter >= 5):

          print("\n Le programme s'est arrété")
          break

        if b in names2:

          if r == "r_has_part":    # Inversion
            a,b = b,a

          if '>' in x:
            dta = getTerm(dfTerm, names)
            x = re.sub("[0-9].*", dta, x)

          print("\n", a, r, b," => Oui, car", a, "r_isa" , x, " et ", x, r, b)
          return

      except:
        pass  

    print(a,r,b,"=>  Aucune inférence inductive trouvée, veuillez avec les autres options. ")
    return

   ############  TRANSITIVITE   ####################

  if (inference_type =='3'):

    if r in [9]:
      a,b = b,a

    dfTerm, dfRel = getData(a,r)
    dfRel= dfRel.sort_values(by='w ', ascending=False)

    IdTerme_2 = np.array(dfRel.node2.values)
    names = dfTerm.set_index("eid").loc[IdTerme_2, ['name']].values.flatten()

    for x in names :

      fileName = getFile(x)
      start = time.time()
      print("\nRecherche... - mot trouvé : ", x)

      if x==b :
        print("\n\n ", a, r, b," => Oui, car", a, r, x, " et ", x, r, b)
        return

      try:

        if (fileName==True):

          print("Le term existe déjà, trouvons un résultat...\n")
          newDfEnt = pd.read_csv(f_Ent, on_bad_lines='skip', delimiter=';')
          newDfRel = pd.read_csv(f_Rel, on_bad_lines='skip', delimiter=';')

        else:
          print("Le term n'existe pas, essayons de le chercher...\n")
          newDfEnt, newDfRel = getData(x, r)

        
        newDfRel = newDfRel.sort_values(by='w ', ascending=False)
        IdTerme_22 = np.array(newDfRel.node2.values)
        names2 = newDfEnt.set_index("eid").loc[IdTerme_22, ['name']].values.flatten()

        end = time.time()
        timer = end-start
        counter += timer

        if(counter >= 5):

          print("\n Le programme s'est arrété")
          break

        if b in names2:

          if r in [9]:

            a,b = b,a

          print("\n", a, r, b," => Oui, car", a, r, x, "et", x, r, b)

          return

      except:
        pass  

    print(a,r,b,"=>  Aucune inférence transitive à distance 2 trouvée ")
    return

Some  particularity

6 => relation "r_isa" : deductive inference.
For example, 'animal' and 'mammifère' are generics of 'chat

8 => relation "r_hypo" : inductive inference.
For example, 'mouche', 'abeille', 'guêpe' are specific of 'insecte'

9 => relation "r_has_part" : transitive inference.
For example, 'voiture' has as parts : 'porte', 'roue', 'moteur'

24 => relation "r_agent-1" : deductive inference.
For example,  'chat' => 'miauler', 'griffer',

28 => relation "r_lieu-1" : transitive inference.
For example, 'Paris' r_lieu-1 'tour Eiffel'

In [ ]:
def main():
  premier = input("Veuillez entrer votre premier terme: \n")

  relationName= input("Veuillez entrer le nom de la relation: \n")

  deuxieme= input("Veuillez entrer votre deuxième terme : \n")

  # get the inference
  print(f"Production d'inférence pour la relation '{relationName}' entre '{premier}' et '{deuxieme}'")

  return getInference(premier, relationName, deuxieme)



###############################################  STARTING... #########################################
main()